In [ ]:
# region imports
from AlgorithmImports import *
# endregion

class BreakOutStrategy(QCAlgorithm):
    # Initialize Algorithm
    def Initialize(self):
    
        self.SetStartDate(2023, 5, 2)
        self.SetEndDate(2024, 7, 24)
        self.SetCash(1000)  
        self.Symbol = self.AddEquity("QQQ", Resolution.Minute).Symbol

        # Strategy Status
        self.first_five_high = None
        self.first_five_low = None
        self.waiting_for_breakout = False
        
        # Program de five first minute candles 
        self.Schedule.On(self.DateRules.EveryDay(self.Symbol), 
                         self.TimeRules.AfterMarketOpen(self.Symbol, 5), 
                         self.SetOpeningRange)

    def SetOpeningRange(self):
        # Reset the range every day
        self.first_five_high = None
        self.first_five_low = None
        self.waiting_for_breakout = True

        # Obtain the candles for the first five minutes once market opens
        history = self.History(self.Symbol, 5, Resolution.Minute)
        
        if not history.empty:
            # Calculate the maxiumum and minimum level
            self.first_five_high = history['high'].max()
            self.first_five_low = history['low'].min()

            self.Debug(f"First 5-minute range set: High={self.first_five_high}, Low={self.first_five_low}")

    def OnData(self, data: Slice):
        if not self.waiting_for_breakout or not data.Bars.ContainsKey(self.Symbol):
            return
        
        # Actutal price
        current_price = data[self.Symbol].Close
        
        # Entry signal
        if current_price > self.first_five_high:
            self.SetHoldings(self.Symbol, 1)  # Entrar en posición larga
            self.Debug(f"Breakout! Entering long at price {current_price}")
            self.waiting_for_breakout = False

        # Stop Loss signal
        if self.Portfolio[self.Symbol].Invested:
            stop_price = self.first_five_low
            if current_price < stop_price:
                self.Liquidate(self.Symbol)
                self.Debug(f"Stop-loss triggered. Liquidating at {current_price}")

    def OnOrderEvent(self, orderEvent):
        if orderEvent.Status == OrderStatus.Filled:
            self.Debug(f"Order filled for {orderEvent.Symbol} at price {orderEvent.FillPrice} at time {self.Time}")
